# DESI Fisher examples

In this notebook we will run some DESI forecasts.

In [1]:
import numpy as np

zedges = np.linspace(1.1, 1.6, 6)
zranges = list(zip(zedges[:-1], zedges[1:]))
nbars = [1079.9998, 1036.0009, 970.9995, 876.9994, 553.9985]  # nobj / deg^2 / dz
b0 = 0.84  # at z = 0

In [2]:
from cosmoprimo.fiducial import DESI
from desilike.theories.galaxy_clustering import (BAOPowerSpectrumTemplate, SimpleBAOWigglesTracerPowerSpectrumMultipoles, DampedBAOWigglesTracerPowerSpectrumMultipoles,
                                                 StandardPowerSpectrumTemplate, KaiserTracerPowerSpectrumMultipoles)
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable, CutskyFootprint, ObservablesCovarianceMatrix
from desilike.likelihoods.galaxy_clustering import ObservablesGaussianLikelihood, SNWeightedPowerSpectrumLikelihood
from desilike import Fisher, setup_logging

setup_logging()

## Let's compare simple isotropic BAO forecasts against some reference

### SN-weighted version

In [3]:
cosmo = DESI()
fo = cosmo.get_fourier()

covariances = []
for zrange, nbar in list(zip(zranges, nbars)):
    z = np.mean(zrange)
    r = 0.5
    sigmaper = (1 - r) * 12.4 * 0.758 * fo.sigma8_z(z, of='delta_cb') / 0.9
    f = fo.sigma8_z(z, of='theta_cb') / fo.sigma8_z(z, of='delta_cb')
    b1 = b0 * fo.sigma8_cb/fo.sigma8_z(z, of='delta_cb')  # prescription for linear bias
    params = {'b1': b1, 'sigmapar': (1. + f) * sigmaper, 'sigmaper': sigmaper}
    # Footprint, to get volume and shot noise; provided nbar is in deg^(-2) dz^(-1)
    footprint = CutskyFootprint(area=14000., zrange=zrange, nbar=nbar * np.diff(zrange), cosmo=cosmo)
    template = BAOPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qisoqap')
    #theory = DampedBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    theory = SimpleBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    likelihood = SNWeightedPowerSpectrumLikelihood(theories=theory, data=params, footprints=footprint, klim=(0.01, 0.5))

    fisher = Fisher(likelihood)
    precision = fisher(**params)
    covariances.append(precision.to_covariance(params=['qiso']))

# Let's compare against reference: within a few percent, not too bad!
refs = [0.7019, 0.7107, 0.7296, 0.7658, 1.0062]
print(100. * np.array([cov.std(params='qiso') for cov in covariances]) / np.array(refs)[:len(covariances)])

[000000.53] [0/1] 01-30 22:38  absl                      INFO     Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
[000000.53] [0/1] 01-30 22:38  absl                      INFO     Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
[000000.53] [0/1] 01-30 22:38  absl                      INFO     Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
[000000.54] [0/1] 01-30 22:38  absl                      INFO     Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
[000000.54] [0/1] 01-30 22:38  absl                      WARNING  No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


/home/adematti/Bureau/DESI/NERSC/cosmodesi/cosmoprimo/cosmoprimo/eisenstein_hu.py:23: UserWarning: EisensteinHuNoWiggleEngine cannot cope with massive neutrinos
  warnings.warn('{} cannot cope with massive neutrinos'.format(self.__class__.__name__))


[000001.52] [0/1] 01-30 22:38  Differentiation           INFO     Varied parameters: ['qiso', 'qap', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1'].
[000001.66] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter qiso.
[000001.68] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter qap.
[000001.71] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000001.73] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000001.75] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al0_-2.
[000001.78] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al0_-1.
[000001.81] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al0_0.
[

[000010.03] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al0_1.
[000010.06] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al2_-3.
[000010.09] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al2_-2.
[000010.12] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al2_-1.
[000010.15] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al2_0.
[000010.18] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al2_1.
[000010.22] [0/1] 01-30 22:38  Differentiation           INFO     Varied parameters: ['qiso', 'qap', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1'].
[000010.26] [0/1] 01-30 22:38  Differentiation           INFO     Using finite-differentiation for parameter 

[000016.05] [0/1] 01-30 22:38  Differentiation           INFO     Using finite-differentiation for parameter qap.
[000016.15] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000016.23] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000016.30] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al0_-2.
[000016.37] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al0_-1.
[000016.45] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al0_0.
[000016.53] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al0_1.
[000016.60] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al2_-3.
[000016.67] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differenti

### Comparison against multipole compression

In [4]:
covariances2 = []
for zrange, nbar in list(zip(zranges, nbars)):
    z = np.mean(zrange)
    r = 0.5
    sigmaper = (1 - r) * 12.4 * 0.758 * fo.sigma8_z(z, of='delta_cb') / 0.9
    f = fo.sigma8_z(z, of='theta_cb') / fo.sigma8_z(z, of='delta_cb')
    params = {'b1': b0 * fo.sigma8_cb/fo.sigma8_z(z, of='delta_cb'), 'sigmapar': (1. + f) * sigmaper, 'sigmaper': sigmaper}
    
    footprint = CutskyFootprint(area=14000., zrange=zrange, nbar=nbar * np.diff(zrange), cosmo=cosmo)
    template = BAOPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qisoqap')
    #theory = DampedBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    theory = SimpleBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    observable = TracerPowerSpectrumMultipolesObservable(data=params,  # data can be a dictionary of parameters
                                                         # fit monopole and quadrupole, between 0.02 and 0.5 h/Mpc, with 0.005 h/Mpc steps
                                                         klim={0: [0.01, 0.5, 0.005], 2: [0.01, 0.5, 0.005], 4: [0.01, 0.5, 0.005]},
                                                         theory=theory)
    covariance = ObservablesCovarianceMatrix(observable, footprints=footprint, resolution=5)  # Gaussian covariance matrix
    likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=covariance(**params))
    fisher = Fisher(likelihood)
    precision = fisher(**params)
    covariances2.append(precision.to_covariance(params=['qiso']))

# multipole compression is very close to optimal...
print(np.array([cov.std(params='qiso') for cov in covariances2]) / np.array([cov.std(params='qiso') for cov in covariances]))

/home/adematti/anaconda3/envs/cosmodesi/lib/python3.9/site-packages/numpy/lib/polynomial.py:1329: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


[000026.42] [0/1] 01-30 22:38  Differentiation           INFO     Varied parameters: ['qiso', 'qap', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1', 'al4_-3', 'al4_-2', 'al4_-1', 'al4_0', 'al4_1'].
[000026.47] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter qiso.
[000026.50] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter qap.
[000026.55] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000026.58] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000026.61] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al0_-2.
[000026.65] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al0_-1.
[000026.68] [0/1] 01-30 22:38  Differentiation           INFO     U

[000038.56] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al0_1.
[000038.61] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al2_-3.
[000038.65] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al2_-2.
[000038.70] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al2_-1.
[000038.75] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al2_0.
[000038.79] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al2_1.
[000038.84] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al4_-3.
[000038.88] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differentiation for parameter al4_-2.
[000038.93] [0/1] 01-30 22:38  Differentiation           INFO     Using auto-differ

[000057.72] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter al4_-1.
[000057.75] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter al4_0.
[000057.79] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter al4_1.
[000057.81] [0/1] 01-30 22:39  Differentiation           INFO     Varied parameters: ['qiso', 'qap', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1', 'al4_-3', 'al4_-2', 'al4_-1', 'al4_0', 'al4_1'].
[000057.84] [0/1] 01-30 22:39  Differentiation           INFO     Using finite-differentiation for parameter qiso.
[000057.88] [0/1] 01-30 22:39  Differentiation           INFO     Using finite-differentiation for parameter qap.
[000057.92] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000057.97] [0/1] 01-30 22:39  Differentiation           INFO    

## BAO + full shape forecasts

In [5]:
cosmo = DESI()
fo = cosmo.get_fourier()

covariances_bao, covariances_bao_highk, covariances_fs_lowk, covariances_bao_fs = [], [], [], []
for zrange, nbar in list(zip(zranges, nbars)):
    z = np.mean(zrange)
    r = 0.5
    sigmaper = (1 - r) * 12.4 * 0.758 * fo.sigma8_z(z, of='delta_cb') / 0.9
    f = fo.sigma8_z(z, of='theta_cb') / fo.sigma8_z(z, of='delta_cb')
    b1 = b0 * fo.sigma8_cb/fo.sigma8_z(z, of='delta_cb')
    params = {'b1': b1, 'sigmapar': (1. + f) * sigmaper, 'sigmaper': sigmaper}
    footprint = CutskyFootprint(area=14000., zrange=zrange, nbar=nbar * np.diff(zrange), cosmo=cosmo)
    template = BAOPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qparqper')
    #theory = DampedBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    theory = SimpleBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    #for param in theory.params.select(basename='al*'): param.update(fixed=True)
    likelihood = SNWeightedPowerSpectrumLikelihood(theories=theory, data=params, footprints=footprint, klim=(0.01, 0.5))
    fisher = Fisher(likelihood)
    precision_bao = fisher(**params)
    covariances_bao.append(precision_bao.to_covariance(params=['qpar', 'qper']))
    
    likelihood.init.update(klim=(0.1, 0.5))
    fisher = Fisher(likelihood)
    precision_bao = fisher(**params)
    covariances_bao_highk.append(precision_bao.to_covariance(params=['qpar', 'qper']))
    
    #likelihood.init.update(klim=(0.01, 0.1))
    params = {'b1': b1}
    template = StandardPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qparqper')
    theory = KaiserTracerPowerSpectrumMultipoles(template=template)
    likelihood = SNWeightedPowerSpectrumLikelihood(theories=theory, data=params, footprints=footprint, klim=(0.01, 0.1))
    fisher = Fisher(likelihood)
    precision_fs = fisher(**params)
    covariances_fs_lowk.append(precision_fs.to_covariance(params=['qpar', 'qper']))
    covariances_bao_fs.append((precision_bao + precision_fs).to_covariance(params=['qpar', 'qper', 'f']))

[000070.53] [0/1] 01-30 22:39  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1'].
[000070.58] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000070.61] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter qper.
[000070.65] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000070.69] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000070.72] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter al0_-2.
[000070.76] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter al0_-1.
[000070.79] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter al0_0.

[000076.33] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter f.
[000076.36] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000076.44] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter sn0.


/home/adematti/Bureau/DESI/NERSC/cosmodesi/desilike/desilike/parameter.py:2735: RuntimeWarning: Mean of empty slice
  new._center = np.nanmean(centers, axis=0)


[000078.84] [0/1] 01-30 22:39  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1'].
[000078.88] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000078.91] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter qper.
[000078.94] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000078.97] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000078.99] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter al0_-2.
[000079.02] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter al0_-1.
[000079.05] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter al0_0.

[000083.52] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter f.
[000083.57] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000083.61] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter sn0.
[000084.46] [0/1] 01-30 22:39  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1'].
[000084.50] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000084.54] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter qper.
[000084.57] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000084.61] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000084.64

[000089.13] [0/1] 01-30 22:39  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'f', 'b1', 'sn0'].
[000089.18] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000089.23] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter qper.
[000089.27] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter f.
[000089.31] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000089.34] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter sn0.
[000090.14] [0/1] 01-30 22:39  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1'].
[000090.19] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[0

[000094.61] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter f.
[000094.62] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000094.64] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter sn0.
[000094.66] [0/1] 01-30 22:39  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'f', 'b1', 'sn0'].
[000094.72] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000094.77] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter qper.
[000094.82] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter f.
[000094.86] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000094.90] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for pa

[000100.19] [0/1] 01-30 22:39  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'f', 'b1', 'sn0'].
[000100.21] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000100.23] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter qper.
[000100.25] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter f.
[000100.26] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000100.28] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter sn0.
[000100.30] [0/1] 01-30 22:39  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'f', 'b1', 'sn0'].
[000100.36] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000100.40] [0/1] 01-30 22:39  Differentiation           INFO     Using auto-different

In [6]:
# 5% - 15% improvement using the full shape < 0.1 h/Mpc
for cov_bao, cov_bao_fs in zip(covariances_bao, covariances_bao_fs):
    print(cov_bao.std('qpar') / cov_bao_fs.std('qpar'), cov_bao.std('qper') / cov_bao_fs.std('qper'))

1.1347383726028908 1.0547740913381625
1.1336187177961294 1.053404415237106
1.1339221050552697 1.0529330266155017
1.1367163869793977 1.0540473509102368
1.166494439812003 1.0734060733926833
